In [1]:
import sys
sys.path.append('../')
import lemur

In [3]:
lemur.full((1,1,1,4,4),100.0)


tensor @ 0x152673bb0
comes_from: [NULL expression]
k:
kernel_tensor @ 0x1338358d0 with length = 16 
stride = [16, 16, 16, 4, 1]
computed = false
shallow = false
tensor([[[[[  0.00,   0.00,   0.00,   0.00],
	   [  0.00,   0.00,   0.00,   0.00],
	   [  0.00,   0.00,   0.00,   0.00],
	   [  0.00,   0.00,   0.00,   0.00]]]]])
shape = [1, 1, 1, 4, 4]
grad:
[NULL kernel_tensor]

In [7]:
a = lemur.LemurTensor(s, requires_grad=True)
b = lemur.LemurTensor(s, requires_grad=True)
c = lemur.LemurTensor(s)
r = lemur.tensor((0,0,0,0,0))

In [4]:
lemur.tensor((0,0.5,0,0,100))

tensor @ 0x12d647b20
comes_from: [NULL expression]
k:
kernel_tensor @ 0x12d647b40 with length = 5 
stride = [5, 5, 5, 5, 1]
computed = false
shallow = false
tensor([[[[[0.0e+00, 5.0e-01, 0.0e+00, 0.0e+00, 1.0e+02]]]]])
shape = [1, 1, 1, 1, 5]
grad:
[NULL kernel_tensor]

In [4]:
d = (a * b) + c
e = (d * d) + d
f = d + e
g = f.sum(r) 
print(g.graph())

└── LemurTensor @ 0x137e3ced0 
    └── [sum]
        └── LemurTensor @ 0x137e35e30 (shape=[5, 5])
            └── [add]
                ├── LemurTensor @ 0x137e3e5c0 (shape=[5, 5])
                │   └── [add]
                │       ├── LemurTensor @ 0x137e33a80 (shape=[5, 5])
                │       │   └── [mul]
                │       │       ├── LemurTensor @ 0x147eb9a30 (shape=[5, 5])
                │       │       └── LemurTensor @ 0x117facf10 (shape=[5, 5])
                │       └── LemurTensor @ 0x117f280b0 (shape=[5, 5])
                └── LemurTensor @ 0x130204330 (shape=[5, 5])
                    └── [add]
                        ├── LemurTensor @ 0x130204280 (shape=[5, 5])
                        │   └── [mul]
                        │       ├── [repeated] LemurTensor @ 0x137e3e5c0 (shape=[5, 5])
                        │       └── [repeated] LemurTensor @ 0x137e3e5c0 (shape=[5, 5])
                        └── [repeated] LemurTensor @ 0x137e3e5c0 (shape=[5, 5])


In [7]:
#%%timeit
d = (a * b) + c
e = (d * d) + d
f = a + b + c + d + e
g = f.sum(r) #fwd takes 5ms
g.backward() #backward takes 15ms

In [5]:
#d.graph()

In [15]:
import torch

In [19]:
at = torch.arange(32., requires_grad=True)
a = at.view(2,4,4)
a.sum().backward()
at.grad


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [13]:
at = torch.empty(s, requires_grad=True)
bt = torch.empty(s, requires_grad=True)
ct = torch.empty(s)


In [ ]:
%%timeit
dt = (at * bt) + ct
et = (dt * dt) + dt
ft = at + bt + ct + dt + et
gt = ft.sum() #fw 3ms
gt.backward() #backward 5ms

4.52 ms ± 311 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
a = lemur.tensor([i for i in range(1,9)], (1,1,2,2,2),  requires_grad=True)
b = lemur.tensor([i for i in range(1+10,9+10)], (1,1,2,2,2),  requires_grad=True)
dm = lemur.tensor([0,0,0,0,0])
v = lemur.tensor([0,1,2,4,3])
a_p = a.permute(v)


In [3]:
%%timeit
c = b * a_p
d = c.sum(dm)
d.backward()


3.27 µs ± 6.94 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
a = lemur.tensor([i for i in range(1,9)], (1,1,2,2,2),  requires_grad=True)
b = lemur.tensor([i for i in range(1+10,9+10)], (1,1,2,2,2),  requires_grad=True)
dm = lemur.tensor([0,0,0,0,0])
v = lemur.tensor([0,1,2,4,3])
a_p = a.permute(v)


In [21]:
a_p = a.permute(v)
print(a)


call
tensor @ 0x13dfa4d30
comes_from: [NULL expression]
k:
kernel_tensor @ 0x13df9bd70 with length = 8 
stride = [8, 8, 4, 2, 1]
computed = false
tensor([[[[[1.0000, 2.0000],[3.0000, 4.0000]],[[5.0000, 6.0000],[7.0000, 8.0000]]]]])
shape = [1, 1, 2, 2, 2]
grad:
kernel_tensor @ 0x13dfb0000 with length = 8 
stride = [8, 8, 4, 2, 1]
computed = false
tensor([[[[[0.0000, 0.0000],[0.0000, 0.0000]],[[0.0000, 0.0000],[0.0000, 0.0000]]]]])
shape = [1, 1, 2, 2, 2]



3.3 µs ± 22.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [19]:
%%timeit
at.permute(0,1,2,4,3).contiguous()

199 µs ± 8.24 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [4]:
import torch
at = torch.tensor([[[[[1.0000, 2.0000],[3.0000, 4.0000]],[[5.0000, 6.0000],[7.0000, 8.0000]]]]],  requires_grad=True, dtype=float)
bt = torch.tensor([[[[[11.0000, 12.0000],[13.0000, 14.0000]],[[15.0000, 16.0000],[17.0000, 18.0000]]]]],  requires_grad=True, dtype=float)
at_p = at.permute([0,1,2,4,3])


In [7]:
%%timeit
ct = bt * at_p
dt = ct.sum()
dt.backward(retain_graph=True)


20.6 µs ± 433 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [6]:
bt.grad

tensor([[[[[1., 3.],
           [2., 4.]],

          [[5., 7.],
           [6., 8.]]]]], dtype=torch.float64)

In [ ]:
x = lemur.tensor([i for i in range(1,33)], (2,2,2,2,2),  requires_grad=True)
y = x.sum(lemur.tensor([0,0,0,0,0]))
y

In [ ]:
y.backward()

In [ ]:
x = lemur.tensor([1.0], requires_grad=True)
y = lemur.tensor([4.0], requires_grad=True)

z = x + y  
w = z * x  
w = w.relu() 

w.backward() 
print("Result from main:", w)

In [36]:
shape = [1,1,1,1,10]

In [ ]:
%%timeit
x = lemur.LemurTensor(shape=shape, requires_grad=False)
y = lemur.LemurTensor(shape=shape, requires_grad=False)
x + y  

In [3]:
import torch 

In [ ]:
%%timeit
x = torch.empty(shape)
y = torch.empty(shape)
x + y  

In [ ]:
t0 = torch.tensor([10.], requires_grad=True)
t1 = torch.tensor([12.], requires_grad=True)
t2 = torch.tensor([-15.], requires_grad=True)
t4 = t0 * t1 + t2
t4


In [12]:
t4.backward()

In [ ]:
t0.grad, t1.grad, t2.grad

In [ ]:
a = lemur.tensor([[10.,12.],[-1.,-4.]], requires_grad = True)
b = lemur.rand_like(a)
c = (a @ b).mean()
c.backward()
a.grad

In [ ]:
c

In [ ]:
a = lemur.tensor([...], requires_grad = True)
b = lemur.tensor([...], requires_grad = True)
c = (a + b).relu()

In [ ]:
x = torch.tensor([[1], [2], [3]])
x.size(), x.stride()

In [ ]:
x.expand(3, 4)


In [ ]:
x.retain_grad()
x.grad

In [ ]:
x = torch.tensor([[1],[2]]).expand((2,10))
x.shape, x.stride()

In [ ]:
y = x.reshape(20)

In [ ]:
y.shape, y.stride

In [ ]:
x = torch.tensor([[10.]]).expand(1,10)
x.requires_grad = True
x.stride()

In [ ]:
x.sum().backward()
x.grad.stride()

In [99]:
x = torch.rand((3,4)).t() #(1,4)
x.requires_grad = True


In [100]:
x.sum().backward()

In [ ]:
x.grad.stride()

In [83]:
float arr[27]
shape (3,3,3) 
stride (9,3,1)

In [ ]:
t0 -> 1,1 
   -> length = 1

t1 -> 1,10 
   -> length = 1   
   -> comesfrom -> t0, expand, 1 10






In [ ]:
x.stride()

In [3]:
import torch

In [ ]:
x = torch.randn((3,1), requires_grad=True)
x2 = torch.randn((3,1), requires_grad=True)

x.stride()

In [ ]:
y = (x*x2).expand(3,4)
y.stride()

In [154]:
y.sum().backward()

In [ ]:
x.grad, x2*4

In [ ]:
torch.empty((1,2,3,4,5)).permute(1,2,3,4,0).stride()


In [ ]:
x = torch.tensor([[1]]).expand(1,10)
x.stride()


In [ ]:
y = x.relu()
y.stride()

In [ ]:
x

In [ ]:
x = torch.tensor([[0.,0.]]).permute(1,0)
x.requires_grad = True
x, x.stride(), x.is_contiguous()


In [217]:
x.sum().backward()

In [ ]:
x.grad.stride(), x.stride()

In [ ]:
x.is_contiguous()

In [ ]:
x.contiguous().stride()

In [ ]:
a = torch.arange(8).view(1,4,2)
a.stride()

In [ ]:
a.permute(1,0).is_contiguous()

In [ ]:
a = torch.arange(1,33, dtype=float, requires_grad=True)
a.view(2,2,2,2,2).sum().backward()
a.grad

In [ ]:
x = lemur.tensor([i for i in range(1,33)], (2,2,2,2,2),  requires_grad=True)
y = x.sum(lemur.tensor([0,0,0,0,0]))
y


In [ ]:
y.backward()

In [ ]:
x

In [ ]:
a.sum(0,keepdim=True).sum(1,keepdim=True).sum(2,keepdim=True).sum(3,keepdim=True)


In [ ]:
a.sum(0,keepdim=True).sum(1,keepdim=True).sum(3,keepdim=True).sum(4,keepdim=True)

In [ ]:
(2,2,2,2,2) -> (1,2,2,2,1)

In [43]:
s = (5,5,50,5,5)
a_t = torch.empty(s)
a_l = lemur.LemurTensor(shape=s, requires_grad=False)



In [ ]:
%%timeit
b = a_t.sum(0,keepdim=True).sum(1,keepdim=True).sum(2,keepdim=True).sum(3,keepdim=True)

In [ ]:
%%timeit
b = a_l.sum(lemur.tensor([0,0,0,0,0])) 

In [ ]:
import torch
torch.tensor([1,2]).expand(10)

In [6]:
print(plot_tensor_graph(d))

AttributeError: module 'lemur' has no attribute 'get_op_name'